# Commerce/Growth Marketing Sample Size Calculator

**Overview:**

This program conducts a sensitivity analysis based on test design parameters to output sample sizes needed and time-to-complete testing in csv format. Users can specify a range of values to conduct power analysis on for hypothesis testing involving differences in means (t-test) and differences in proportions (z-test).

**Instructions:**

1.   Input test design parameters in "Test Design Parameters"
2.   Go to Runtime and Run All
3.   See auto-generated spreadsheet in File tab!


**Parameter Definitions:**

*   test_type = determines if you are testing a difference in proportions or a difference in means
*   desired_alpha = the statistical significance level you would like for your test (Probability of Type 1 Error)
*   desired_stat_power = the statistical power you would like for your test (1 - Probability of Type 2 Error)
*   number_of_groups = # of groups that will be in your experiment, including control group (minimum of 2; 1 treatment and 1 control)
*   metric_name = name of metric you are conducting power analysis for; used for naming output csv
*   mde_relative_lower_bound_percent = minimum detectable effect lower bound you would like to conduct sensitivity analysis on, expressed as a percent relative lift. For example: input of .5 equals .5%
*   mde_relative_upper_bound_percent = minimum detectable effect upper bound you would like to conduct sensitivity analysis on, expressed as a percent relative lift. For example: input of 1 equals 1%
*   mde_relative_step_percent = step size you would like between lower and upper bound for sensitivity analysis. For example: input of 0.3 equals 0.3%

*   baseline_stat = baseline proportion for proportions or baseline mean for means
*   baseline_standard_deviation = baseline standard deviation, used only for testing difference of means (test_type = means)
*  expected_total_weekly_traffic = total weekly traffic we expect to receive for our experiment (inclusive of control and treatment groups)


**Notes:**
*   Calculations assume even traffic split of expected_total_weekly_traffic across number_of_groups. For cases where we expect to allocate only a portion of expected_total_weekly_traffic, please adjust your expected_total_weekly_traffic input accordingly


*   This program utilizes the following methods to calculate sample size: https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html
https://www.statsmodels.org/stable/generated/statsmodels.stats.power.zt_ind_solve_power.html







In [ ]:
# @title Test Design Parameters { run: "auto", vertical-output: true, form-width: "50%", display-mode: "both" }
test_type = "proportions" # @param ["proportions", "means"]
desired_stat_power = 0.8 # @param {type:"slider", min:0.6, max:1, step:0.025}

desired_alpha = 0.1 # @param {type:"slider", min:0.01, max:0.50, step:0.005}
number_of_groups = 2 # @param {type:"slider", min:2, max:15, step:1}
metric_name = 'roadblock conversion rate' #@param {type:"string"}

mde_relative_lower_bound_percent = .25 # @param {type:"number"}
mde_relative_upper_bound_percent = 3 # @param {type:"number"}
mde_relative_step_percent = .25 # @param {type:"number"}

baseline_stat_lower_bound = .135 # @param {type:"number"}
baseline_stat_upper_bound = .16 # @param {type:"number"}
baseline_stat_step = 0.01 # @param {type:"number"}

baseline_standard_deviation = 1.5 # @param {type:"number"}

expected_total_weekly_traffic = 133538 # @param {type:"number"}


In [ ]:
# @title Function Definitions and Calls
# import packages and necessary functions
import timeit
import scipy
import numpy as np
import pandas as pd
import math
from datetime import datetime
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

current_d = str(datetime.now())
current_d = current_d[:current_d.find('.')]
np.random.seed(42)

# function to check user parameters to make sure they are valid.
def validate_test_parameters():
  is_valid = True
  # make sure upper bound is greater than lower bound
  if mde_relative_lower_bound_percent > mde_relative_upper_bound_percent:
    print("ERROR: Your mde_relative_lower_bound_percent is greater than your mde_relative_upper_bound_percent")
    is_valid = False
    #make sure step size + lower bound is not greater upper bound
  elif mde_relative_lower_bound_percent + mde_relative_step_percent > mde_relative_upper_bound_percent:
    print("ERROR: Your mde_relative_step_percent is too high")
    is_valid = False
  if baseline_stat_lower_bound > baseline_stat_upper_bound:
    print("ERROR: Your baseline_stat_lower_bound is greater than your baseline_stat_upper_bound")
    is_valid = False
  elif baseline_stat_lower_bound + baseline_stat_step > baseline_stat_upper_bound:
    print("ERROR: Your baseline_stat_step is too high")
    is_valid = False
  if baseline_stat_step <= 0:
    print("ERROR: Your baseline_stat_step is not greater than 0")
    is_valid = False
  if mde_relative_step_percent <= 0:
    print("ERROR: Your mde_relative_step_percent is not greater than 0")
    is_valid = False
  # make sure expected total_weekly_traffic is greater than 0
  if expected_total_weekly_traffic <= 0:
    print("ERROR: Your expected total weekly traffic is not greater than 0")
    is_valid = False
  return(is_valid)

#generator function to create ranges for sensitivity analysis
def frange(x,y,jump):
  while x <= y+ jump:
    yield round(x,7)
    x+= jump

# takes in a dictionary of test design parameters and a validated test parameter boolean and returns a mde-sensitivy analysis in csv format of sample sizes
def sample_size(vtp_boolean):
  if vtp_boolean == False:
    print("\nExiting program, please re-run with valid parameters")
    pass
  else:
    print("\nIt looks like we have our test design parameters all set, let's conduct our sample sizing sensitivity analysis!")
    set_alpha = desired_alpha
    mde_list = list(frange(mde_relative_lower_bound_percent, mde_relative_upper_bound_percent, mde_relative_step_percent))
    baseline_stat_list = list(frange(baseline_stat_lower_bound, baseline_stat_upper_bound, baseline_stat_step))

    if test_type == 'means': # if we are testing difference of means
      ss_df = pd.DataFrame(columns = ['sample_size_per_group','mde', 'alpha', 'power', 'baseline_mean', 'expected_total_weekly_traffic', 'number_of_groups', 'baseline_standard_deviation'])
      ttc_df = pd.DataFrame(columns = ['expected weeks to reach stat-sig','mde', 'alpha', 'power', 'baseline_mean', 'expected_total_weekly_traffic', 'number_of_groups', 'baseline_standard_deviation'])
      il_df = pd.DataFrame(columns = ['incremental lift','mde', 'alpha', 'power', 'baseline_mean', 'expected_total_weekly_traffic', 'number_of_groups', 'baseline_standard_deviation'])
      for m in mde_list:
        for s in baseline_stat_list:
          effect = 0.01* m*s/baseline_standard_deviation

          ss = tt_ind_solve_power(effect_size = effect, alpha = set_alpha, \
                               power = desired_stat_power, ratio =1, alternative = 'two-sided')
          ttc = ss * number_of_groups/expected_total_weekly_traffic
          il = 0.01*m* s * expected_total_weekly_traffic
          # row inputs must match column inputs in pd.DataFrame() instantiation
          ss_row = [int(round(ss)), m, set_alpha, desired_stat_power, s, expected_total_weekly_traffic, number_of_groups, baseline_standard_deviation]
          ttc_row = [round(ttc,2), m, set_alpha, desired_stat_power, s, expected_total_weekly_traffic, number_of_groups, baseline_standard_deviation]
          il_row = [int(round(il)), m, set_alpha, desired_stat_power, s, expected_total_weekly_traffic, number_of_groups, baseline_standard_deviation]
          ss_df.loc[len(ss_df)] = ss_row
          ttc_df.loc[len(ttc_df)] = ttc_row
          il_df.loc[len(il_df)] = il_row
      ss_wide=pd.pivot(ss_df, index=['alpha','power', 'baseline_mean', 'baseline_standard_deviation', 'expected_total_weekly_traffic', 'number_of_groups'], columns = 'mde',values = 'sample_size_per_group') #Reshape from long to wide
      ttc_wide=pd.pivot(ttc_df, index=['alpha','power', 'baseline_mean','baseline_standard_deviation', 'expected_total_weekly_traffic', 'number_of_groups', 'baseline_standard_deviation'], columns = 'mde',values = 'expected weeks to reach stat-sig') #Reshape from long to wide
      il_wide=pd.pivot(il_df, index=['alpha','power', 'baseline_mean','baseline_standard_deviation', 'expected_total_weekly_traffic', 'number_of_groups', 'baseline_standard_deviation'], columns = 'mde',values = 'incremental lift') #Reshape from long to wide
      ss_csv_name =  current_d+ '_'+ metric_name + "_2sample_mean_ttest_sample_size_analysis.csv"
      ttc_csv_name = current_d+ '_'+ metric_name + "_2sample_mean_ttest_test_time_analysis.csv"
      il_csv_name = current_d+ '_'+ metric_name + "_2sample_mean_ttest_incremental_weekly_lift_analysis.csv"

    else: # if we are testing difference of proportions
      ss_df = pd.DataFrame(columns = ['sample_size_per_group','mde', 'alpha', 'power', 'baseline_prop', 'expected_total_weekly_traffic', 'number_of_groups'])
      ttc_df = pd.DataFrame(columns = ['expected weeks to reach stat-sig','mde', 'alpha', 'power', 'baseline_prop', 'expected_total_weekly_traffic', 'number_of_groups'])

      il_df = pd.DataFrame(columns = ['incremental lift','mde', 'alpha', 'power', 'baseline_prop', 'expected_total_weekly_traffic', 'number_of_groups'])
      for m in mde_list:
        for s in baseline_stat_list:
          effect = (0.01*m*s)/ ((s * (1-s))**0.5)

          ss = zt_ind_solve_power(effect_size = effect, alpha = set_alpha, \
                               power = desired_stat_power, ratio =1, alternative = 'two-sided')

          ttc = ss * number_of_groups/expected_total_weekly_traffic
          il = 0.01*m* s * expected_total_weekly_traffic
          # row inputs must match column inputs in pd.DataFrame() instantiation
          ss_row = [int(round(ss)), m, set_alpha, desired_stat_power, s, expected_total_weekly_traffic, number_of_groups]
          ttc_row = [round(ttc,2), m, set_alpha, desired_stat_power, s, expected_total_weekly_traffic, number_of_groups]
          il_row = [int(round(il)), m, set_alpha, desired_stat_power, s, expected_total_weekly_traffic, number_of_groups]
          ss_df.loc[len(ss_df)] = ss_row
          ttc_df.loc[len(ttc_df)] = ttc_row
          il_df.loc[len(il_df)] = il_row
      ss_wide=pd.pivot(ss_df, index=['alpha','power', 'baseline_prop', 'expected_total_weekly_traffic','number_of_groups'], columns = 'mde',values = 'sample_size_per_group') #Reshape from long to wide
      ttc_wide=pd.pivot(ttc_df, index=['alpha','power', 'baseline_prop', 'expected_total_weekly_traffic','number_of_groups'], columns = 'mde',values = 'expected weeks to reach stat-sig') #Reshape from long to wide
      il_wide=pd.pivot(il_df, index=['alpha','power', 'baseline_prop', 'expected_total_weekly_traffic','number_of_groups'], columns = 'mde',values = 'incremental lift') #Reshape from long to wide
      ss_csv_name = current_d+ '_'+ metric_name + "_2sample_proportion_ztest_sample_size_analysis.csv"
      ttc_csv_name = current_d+ '_'+ metric_name + "_2sample_proportion_ztest_test_time_analysis.csv"
      il_csv_name = current_d+ '_'+ metric_name + "_2sample_proportion_ztest_incremental_weekly_lift_analysis.csv"

    ss_wide.to_csv(ss_csv_name) #write analysis to csv
    ttc_wide.to_csv(ttc_csv_name) #write analysis to csv
    il_wide.to_csv(il_csv_name) #write analysis to csv
    print("\nAnalysis completed! Please see csv for required sample size  for each group and weeks-to-test. \n\n\
    Note: This program utilizes a two sample z-test for differences in proportions and a two sample t-test for a difference in means")
start = datetime.now()
vtp = validate_test_parameters()
sample_size(vtp)
end = datetime.now()
ttc = end - start
print("\nExecution time: " + str(ttc))



It looks like we have our test design parameters all set, let's conduct our sample sizing sensitivity analysis!

Analysis completed! Please see csv for required sample size  for each group and weeks-to-test. 

    Note: This program utilizes a two sample z-test for differences in proportions and a two sample t-test for a difference in means

Execution time: 0:00:00.691406


In [ ]:
# @title Code to Refresh Directory
#code to delete all files in directory
import os
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".csv"):
        file_path = os.path.join(os.getcwd(), filename)
        #os.remove(file_path) #uncomment to delete csv files in current directory (start over)